# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

import statsmodels.api as sm

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [4]:
# Completando as células da coluna tempo_emprego com a média de seus valores
df['tempo_emprego'] = df['tempo_emprego'].fillna(df['tempo_emprego'].mean())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          15000 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [5]:
# Separando as variáveis numéricas (num) e categóricas (cat), reduzindo a base as dados de interesse para o estudo:
cat = ['sexo','tipo_renda','educacao','estado_civil','tipo_residencia']
num = ['idade', 'tempo_emprego', 'qtd_filhos']

In [6]:
# Variáveis independentes, de interesse
X = df[cat + num]
# Variável dependente
y = df['renda']

In [7]:
# Convertendo todas as colunas booleanas para 0 e 1 automaticamente
bool_cols = X.select_dtypes(include=['bool']).columns
for col in bool_cols:
    X[col] = X[col].replace({False: 0, True: 1}).astype(int)

In [8]:
# Retirando as colunas tipo object
X = pd.get_dummies(X, columns=cat, drop_first=True)
X = X.astype(float).dropna()

In [9]:
# Garantindo que as colunas numéricas sejam float
for col in num:
    X[col] = pd.to_numeric(X[col], errors='coerce')
X[num] = X[num].fillna(X[num].mean())

# Garantindo que y seja numérico
y = pd.to_numeric(y, errors='coerce').fillna(y.mean())

# Escalando as variáveis numéricas
scaler = StandardScaler()
X[num] = scaler.fit_transform(X[num])

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Tamanho das bases
print(f"Tamanho do conjunto de treinamento: {X_train.shape[0]} linhas")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]} linhas")

Tamanho do conjunto de treinamento: 11250 linhas
Tamanho do conjunto de teste: 3750 linhas


2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [11]:
# Lista de alphas para Ridge
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

In [12]:
# Rodando Ridge e avaliando R² na base de testes
print("Resultados Ridge na base de testes:")
best_alpha_r = None
best_r2_r = -float('inf')  # Inicializando com valor muito baixo

for alpha in alphas:
    if alpha == 0:
        model = LinearRegression()  # Alpha = 0 é equivalente a regressão linear
    else:
        model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print(f'Alpha: {alpha}, R²: {r2:.4f}')
    
    # Identificando o melhor modelo
    if r2 > best_r2_r:
        best_r2_r = r2
        best_alpha_r = alpha

Resultados Ridge na base de testes:
Alpha: 0, R²: 0.2682
Alpha: 0.001, R²: 0.2682
Alpha: 0.005, R²: 0.2682
Alpha: 0.01, R²: 0.2682
Alpha: 0.05, R²: 0.2682
Alpha: 0.1, R²: 0.2682


In [13]:
# Resultado do melhor modelo
print(f'\nO melhor resultado encontrado no método de regularização ridge é para alpha {best_alpha_r}. Já que ali se encontra maiores valores de R² = {best_r2_r:.4f}')


O melhor resultado encontrado no método de regularização ridge é para alpha 0.1. Já que ali se encontra maiores valores de R² = 0.2682


3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?

In [14]:
print("\nResultados Lasso na base de testes:")
best_alpha_l = None
best_r2_l = -float('inf')

for alpha in alphas:
    if alpha == 0:
        model = LinearRegression()
    else:
        model = Lasso(alpha=alpha, max_iter=10000) 
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print(f'Alpha: {alpha}, R²: {r2:.4f}')
    
    if r2 > best_r2_l:
        best_r2_l = r2
        best_alpha_l = alpha


Resultados Lasso na base de testes:
Alpha: 0, R²: 0.2682
Alpha: 0.001, R²: 0.2682
Alpha: 0.005, R²: 0.2682
Alpha: 0.01, R²: 0.2682
Alpha: 0.05, R²: 0.2682
Alpha: 0.1, R²: 0.2682


In [15]:
print(f'\nO melhor resultado encontrado no método de regularização lasso é para alpha {best_alpha_l}, já que ali se encontra o maior valor de R² = {best_r2_l:.4f}')


O melhor resultado encontrado no método de regularização lasso é para alpha 0.1, já que ali se encontra o maior valor de R² = 0.2682


4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?

In [16]:
# Função para seleção stepwise
def stepwise_selection(X, y, threshold_in=0.05, threshold_out=0.10, verbose=True):
    included = []
    while True:
        changed = False
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=float)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(X[included + [new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Adicionada: {best_feature}, p-valor = {best_pval:.4f}')
        
        if included:
            model = sm.OLS(y, sm.add_constant(X[included])).fit()
            pvalues = model.pvalues.iloc[1:]  # Ignorar o intercepto
            worst_pval = pvalues.max()
            if worst_pval > threshold_out:
                worst_feature = pvalues.idxmax()
                included.remove(worst_feature)
                changed = True
                if verbose:
                    print(f'Removida: {worst_feature}, p-valor = {worst_pval:.4f}')
        
        if not changed:
            break
    return included

In [17]:
# Aplicando stepwise
print("\nSeleção Stepwise no conjunto de treino:")
selected_features = stepwise_selection(X_train, y_train)
print("\nVariáveis selecionadas:", selected_features)

# Treinando e avaliando o modelo stepwise
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]
model = LinearRegression()
model.fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)
r2_s = r2_score(y_test, y_pred)
print(f'\nR² na base de testes com modelo stepwise: {r2_s:.4f}')


Seleção Stepwise no conjunto de treino:
Adicionada: tempo_emprego, p-valor = 0.0000
Adicionada: sexo_M, p-valor = 0.0000
Adicionada: tipo_renda_Empresário, p-valor = 0.0000
Adicionada: educacao_Superior completo, p-valor = 0.0000
Adicionada: tipo_renda_Pensionista, p-valor = 0.0000
Adicionada: idade, p-valor = 0.0000
Adicionada: estado_civil_Solteiro, p-valor = 0.0236

Variáveis selecionadas: ['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'educacao_Superior completo', 'tipo_renda_Pensionista', 'idade', 'estado_civil_Solteiro']

R² na base de testes com modelo stepwise: 0.2680


5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

In [18]:
# Comparação final
print("\nComparação entre os métodos:")
print(f"Melhor R² Ridge: {best_r2_r:.4f} (Alpha = {best_alpha_r})")
print(f"Melhor R² Lasso: {best_r2_l:.4f} (Alpha = {best_alpha_l})")
print(f"R² Stepwise: {r2_s:.4f}")

best_method = max([('Ridge', best_r2_r, best_alpha_r), 
                   ('Lasso', best_r2_l, best_alpha_l), 
                   ('Stepwise', r2_s, None)], 
                  key=lambda x: x[1])

print(f'\nMelhor resultado geral: {best_method[0]} com R² = {best_method[1]:.4f}')
if best_method[0] != 'Stepwise':
    print(f'Alpha do melhor modelo: {best_method[2]}')


Comparação entre os métodos:
Melhor R² Ridge: 0.2682 (Alpha = 0.1)
Melhor R² Lasso: 0.2682 (Alpha = 0.1)
R² Stepwise: 0.2680

Melhor resultado geral: Lasso com R² = 0.2682
Alpha do melhor modelo: 0.1


6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

In [19]:
# Criando relação entre variáveis
# 1. Interações
X['idade_x_qtd_filhos'] = X['idade'] * X['qtd_filhos']

# 2. Razão
X['tempo_emprego_por_idade'] = X['tempo_emprego'] / X['idade'].replace(0, 1e-6)

# Atualizando a lista de variáveis numéricas com as novas features
num_ext = num + ['idade_x_qtd_filhos', 'tempo_emprego_por_idade']

In [20]:
X[num_ext] = X[num_ext].fillna(X[num_ext].mean())
# Garantindo que y seja numérico
y = pd.to_numeric(y, errors='coerce').fillna(y.mean())

# Escalando as variáveis numéricas
scaler = StandardScaler()
X[num_ext] = scaler.fit_transform(X[num_ext])

# Separando em treinamento (75%) e teste (25%)
X_train, X_test, y_models_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [21]:
# Lasso
print("\nResultados Lasso na base de testes:")
best_r2_lasso = -float('inf')
best_alpha_lasso = None
for alpha in alphas:
    if alpha == 0:
        model = LinearRegression()
    else:
        model = Lasso(alpha=alpha, max_iter=10000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print(f'Alpha: {alpha}, R²: {r2:.4f}')
    if r2 > best_r2_lasso:
        best_r2_lasso = r2
        best_alpha_lasso = alpha


Resultados Lasso na base de testes:
Alpha: 0, R²: 0.2682
Alpha: 0.001, R²: 0.2682
Alpha: 0.005, R²: 0.2682
Alpha: 0.01, R²: 0.2682
Alpha: 0.05, R²: 0.2682
Alpha: 0.1, R²: 0.2682


7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [22]:
# Ajustando a árvore de regressão com diferentes hiperparâmetros
print("\nResultados da Árvore de Regressão na base de testes:")
best_r2_tree = -float('inf')
best_params = None

# Testando combinações de profundidade máxima e mínimo de amostras por folha
max_depths = [None, 5, 10, 15]  # None = sem limite, 5, 10, 15 = profundidades específicas
min_samples_leafs = [1, 5, 10]   # Mínimo de amostras por folha

for max_depth in max_depths:
    for min_samples_leaf in min_samples_leafs:
        model = DecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        print(f'max_depth: {max_depth}, min_samples_leaf: {min_samples_leaf}, R²: {r2:.4f}')
        if r2 > best_r2_tree:
            best_r2_tree = r2
            best_params = (max_depth, min_samples_leaf)

# Resultado da melhor árvore
print(f'\nMelhor Árvore de Regressão: max_depth = {best_params[0]}, min_samples_leaf = {best_params[1]}')
print(f'R² na base de testes com a melhor árvore: {best_r2_tree:.4f}')

# Comparando com os modelos anteriores
r2_stepwise = 0.2680  # Do seu resultado anterior
print("\nComparação com modelos anteriores:")
print(f"R² Stepwise: {r2_stepwise:.4f}")
print(f"R² Melhor Árvore: {best_r2_tree:.4f}")

if best_r2_tree > r2_stepwise:
    print(f"\nA árvore de regressão melhorou o R² em relação ao Stepwise!")
else:
    print(f"\nA árvore de regressão não melhorou o R² em relação ao Stepwise.")


Resultados da Árvore de Regressão na base de testes:
max_depth: None, min_samples_leaf: 1, R²: 0.3005
max_depth: None, min_samples_leaf: 5, R²: 0.3155
max_depth: None, min_samples_leaf: 10, R²: 0.3628
max_depth: 5, min_samples_leaf: 1, R²: 0.3646
max_depth: 5, min_samples_leaf: 5, R²: 0.3505
max_depth: 5, min_samples_leaf: 10, R²: 0.3508
max_depth: 10, min_samples_leaf: 1, R²: 0.3346
max_depth: 10, min_samples_leaf: 5, R²: 0.3118
max_depth: 10, min_samples_leaf: 10, R²: 0.3665
max_depth: 15, min_samples_leaf: 1, R²: 0.3259
max_depth: 15, min_samples_leaf: 5, R²: 0.3218
max_depth: 15, min_samples_leaf: 10, R²: 0.3646

Melhor Árvore de Regressão: max_depth = 10, min_samples_leaf = 10
R² na base de testes com a melhor árvore: 0.3665

Comparação com modelos anteriores:
R² Stepwise: 0.2680
R² Melhor Árvore: 0.3665

A árvore de regressão melhorou o R² em relação ao Stepwise!
